# Mecabを用いた形態素解析によるクリーニング

In [1]:
# 必要なライブラリをインポート
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import json

# Jupyter Notebook用の設定
%matplotlib inline

In [2]:
# データのパス
data_path = '/Users/manqueenmannequin/TW_otameshi/project_root/data/processed/android_cleaned.csv'

# データの読み込み
df = pd.read_csv(data_path)

# データの基本情報を確認
print(f"データサイズ: {df.shape}")
df.head()

データサイズ: (1000, 16)


,id,userName,userImage,date,score,scoreText,url,title,text,replyDate,replyText,version,thumbsUp,criterias,positive_review,negative_review
0,9e31e6b7-605a-4b6b-9146-5a17644ba6b8,シュン,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T14:01:42.496Z,1,1,https://play.google.com/store/apps/details?id=...,NaN,インストール済みアプリの広告動画で✕ボタンを押しそこねた場合、そのアプリが起動するのが凄く邪...,2024-09-16T08:11:04.465Z,いつもアプリをお使いいただき、ありがとうございます。\n動画広告について、貴重なご意見を頂き...,9.6.2,70,[],特になし,- インストール済みアプリの広告動画で✕ボタンを押しそこねるとそのアプリが起動し、これが非常...
1,052823d1-61ed-4374-8a70-348fd02cd274,Asami Shikata,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T11:00:06.814Z,3,3,https://play.google.com/store/apps/details?id=...,NaN,9.6.2 指摘した問題案件の改善されず。( 9.5.0 地図は表示される様になった、でも、...,2024-10-29T06:47:35.118Z,いつもアプリをお使いいただき、ありがとうございます！\n地図のリストをタップしても非表示にな...,9.6.2,1,[{'criteria': 'vaf_never_display_disruptive_ad...,特になし,- 指摘した問題が改善されていない
2,2bd1806d-3415-4ac6-9db2-70800303fc48,酒井雄太,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-22T21:54:44.660Z,4,4,https://play.google.com/store/apps/details?id=...,NaN,Wi-Fiがある場所で自動的に接続できるので便利だと思います。,NaN,NaN,NaN,0,[],Wi-Fiがある場所で自動的に接続できるので便利。,特になし。
3,62bedb1c-1dd6-4807-8544-a04ceda38826,菅原金二郎,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:53:26.341Z,2,2,https://play.google.com/store/apps/details?id=...,NaN,CMが多い ５秒から30秒動画が流れます,NaN,NaN,9.6.2,0,[],特になし,CMが多く、5秒から30秒の動画が流れる
4,f67f4c45-f19c-4f7b-b792-6c703bb05915,みかん,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:23:41.626Z,5,5,https://play.google.com/store/apps/details?id=...,NaN,モバイルスマホを使っていますが、ネット回線が無くてもGMOWiFiで問題なく 検索やネット回...,2024-11-25T01:08:32.912Z,いつもアプリをお使いいただき、ありがとうございます！\n\n外出時のWiFi利用にお役立てい...,9.6.2,0,[],ネット回線が無くてもGMOWiFiで問題なく検索やネット回線が使用できるところ。,特になし。


In [3]:
# データのpositive_review, negative_reviewが「特になし」または「特になし。」を含む場合は，その欄を「-」に変換

df['positive_review'] = df['positive_review'].fillna('').apply(lambda x: '-' if '特になし' in x else x)
df['negative_review'] = df['negative_review'].fillna('').apply(lambda x: '-' if '特になし' in x else x)

# データのpositive_review, negative_reviewが欠損している or 空白の場合は，その欄を「-」に変換

df['positive_review'] = df['positive_review'].replace('', '-')
df['negative_review'] = df['negative_review'].replace('', '-')

df.head()

,id,userName,userImage,date,score,scoreText,url,title,text,replyDate,replyText,version,thumbsUp,criterias,positive_review,negative_review
0,9e31e6b7-605a-4b6b-9146-5a17644ba6b8,シュン,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T14:01:42.496Z,1,1,https://play.google.com/store/apps/details?id=...,NaN,インストール済みアプリの広告動画で✕ボタンを押しそこねた場合、そのアプリが起動するのが凄く邪...,2024-09-16T08:11:04.465Z,いつもアプリをお使いいただき、ありがとうございます。\n動画広告について、貴重なご意見を頂き...,9.6.2,70,[],-,- インストール済みアプリの広告動画で✕ボタンを押しそこねるとそのアプリが起動し、これが非常...
1,052823d1-61ed-4374-8a70-348fd02cd274,Asami Shikata,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T11:00:06.814Z,3,3,https://play.google.com/store/apps/details?id=...,NaN,9.6.2 指摘した問題案件の改善されず。( 9.5.0 地図は表示される様になった、でも、...,2024-10-29T06:47:35.118Z,いつもアプリをお使いいただき、ありがとうございます！\n地図のリストをタップしても非表示にな...,9.6.2,1,[{'criteria': 'vaf_never_display_disruptive_ad...,-,- 指摘した問題が改善されていない
2,2bd1806d-3415-4ac6-9db2-70800303fc48,酒井雄太,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-22T21:54:44.660Z,4,4,https://play.google.com/store/apps/details?id=...,NaN,Wi-Fiがある場所で自動的に接続できるので便利だと思います。,NaN,NaN,NaN,0,[],Wi-Fiがある場所で自動的に接続できるので便利。,-
3,62bedb1c-1dd6-4807-8544-a04ceda38826,菅原金二郎,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:53:26.341Z,2,2,https://play.google.com/store/apps/details?id=...,NaN,CMが多い ５秒から30秒動画が流れます,NaN,NaN,9.6.2,0,[],-,CMが多く、5秒から30秒の動画が流れる
4,f67f4c45-f19c-4f7b-b792-6c703bb05915,みかん,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:23:41.626Z,5,5,https://play.google.com/store/apps/details?id=...,NaN,モバイルスマホを使っていますが、ネット回線が無くてもGMOWiFiで問題なく 検索やネット回...,2024-11-25T01:08:32.912Z,いつもアプリをお使いいただき、ありがとうございます！\n\n外出時のWiFi利用にお役立てい...,9.6.2,0,[],ネット回線が無くてもGMOWiFiで問題なく検索やネット回線が使用できるところ。,-


In [4]:
!pip show mecab-python3

Name: mecab-python3
Version: 1.0.10
Summary: Python wrapper for the MeCab morphological analyzer for Japanese
Home-page: https://github.com/SamuraiT/mecab-python3
Author: 
Author-email: 
License: BSD
Location: /Users/manqueenmannequin/Library/Caches/pypoetry/virtualenvs/project-root-rWJ-GWg1-py3.12/lib/python3.12/site-packages
Requires: 
Required-by: 


In [5]:
import MeCab
print(MeCab.VERSION)


0.996


In [6]:
import MeCab

# mecabrcのパスを明示的に指定
mecab = MeCab.Tagger("-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd -r /opt/homebrew/etc/mecabrc")

# テキスト解析
print(mecab.parse("形態素解析を試してみます。"))


形態素解析	名詞,固有名詞,一般,*,*,*,形態素解析,ケイタイソカイセキ,ケイタイソカイセキ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
試し	動詞,自立,*,*,五段・サ行,連用形,試す,タメシ,タメシ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
み	動詞,非自立,*,*,一段,連用形,みる,ミ,ミ
ます	助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス
。	記号,句点,*,*,*,*,。,。,。
EOS



In [7]:
review_document = 'このサイズ感が素晴らしいです。'
ochasen_tagger = MeCab.Tagger("-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd -r /opt/homebrew/etc/mecabrc")
node = ochasen_tagger.parse(review_document)

print(node)

この	連体詞,*,*,*,*,*,この,コノ,コノ
サイズ	名詞,一般,*,*,*,*,サイズ,サイズ,サイズ
感	名詞,接尾,一般,*,*,*,感,カン,カン
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
素晴らしい	形容詞,自立,*,*,形容詞・イ段,基本形,素晴らしい,スバラシイ,スバラシイ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。
EOS



In [8]:
def lemmatize_entities(entity_list, ochasen_tagger):
    for i, entity in enumerate(entity_list):
        node = ochasen_tagger.parse(entity)
        lines = node.split("\n")
        for line in lines[:-2]:  # 最後のEOSや空行を無視
            fields = line.split("\t")
            print(fields)  # デバッグ用
            
            if len(fields) >= 2:
                # fields[1]をカンマで分割
                features = fields[1].split(",")
                # features[6]が原形（基本形）
                if len(features) > 6:
                    entity_list[i] = features[6]
    return entity_list

# 入力リスト
entity_list = ['強い', '強く', '強']

# 関数の実行
lemmatized_list = lemmatize_entities(entity_list, ochasen_tagger)
print("結果:", lemmatized_list)


['強い', '形容詞,自立,*,*,形容詞・アウオ段,基本形,強い,ツヨイ,ツヨイ']
['強く', '形容詞,自立,*,*,形容詞・アウオ段,連用テ接続,強い,ツヨク,ツヨク']
['強', '形容詞,自立,*,*,形容詞・アウオ段,ガル接続,強い,ツヨ,ツヨ']
結果: ['強い', '強い', '強い']


In [9]:
def extract_keywords(review_document, ochasen_tagger):
    review_keywords_list = []
    node = ochasen_tagger.parse(review_document)
    lines = node.split("\n")

    for line in lines:
        if line == "EOS" or line.strip() == "":
            continue
        fields = line.split("\t")
        if len(fields) < 2:
            continue

        pos_info = fields[1].split(",")
        pos_major = pos_info[0]  # 名詞,動詞,形容詞など
        pos_sub1 = pos_info[1]   # 固有名詞,一般 などの細分類

        if pos_major in ["名詞", "形容詞", "動詞"]:
            review_keywords_list.append(fields[0])

    return review_keywords_list

# Example usage
review_document = 'インストール済みアプリの広告動画で✕ボタンを押しそこねるとそのアプリが起動し、これが非常に邪魔。'
keywords = extract_keywords(review_document, ochasen_tagger)
print(keywords)


['インストール', '済み', 'アプリ', '広告', '動画', 'ボタン', '押し', 'そこねる', 'アプリ', '起動', 'し', 'これ', '非常', '邪魔']


In [10]:
# データのpositive_review, negative_reviewに対して形態素解析を実行，結果を新たなカラムとして追加

df['positive_review_keywords'] = df['positive_review'].apply(lambda x: extract_keywords(x, ochasen_tagger))
df['negative_review_keywords'] = df['negative_review'].apply(lambda x: extract_keywords(x, ochasen_tagger))

df.head()

,id,userName,userImage,date,score,scoreText,url,title,text,replyDate,replyText,version,thumbsUp,criterias,positive_review,negative_review,positive_review_keywords,negative_review_keywords
0,9e31e6b7-605a-4b6b-9146-5a17644ba6b8,シュン,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T14:01:42.496Z,1,1,https://play.google.com/store/apps/details?id=...,NaN,インストール済みアプリの広告動画で✕ボタンを押しそこねた場合、そのアプリが起動するのが凄く邪...,2024-09-16T08:11:04.465Z,いつもアプリをお使いいただき、ありがとうございます。\n動画広告について、貴重なご意見を頂き...,9.6.2,70,[],-,- インストール済みアプリの広告動画で✕ボタンを押しそこねるとそのアプリが起動し、これが非常...,[],"[インストール, 済み, アプリ, 広告, 動画, ボタン, 押し, そこねる, アプリ, ..."
1,052823d1-61ed-4374-8a70-348fd02cd274,Asami Shikata,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T11:00:06.814Z,3,3,https://play.google.com/store/apps/details?id=...,NaN,9.6.2 指摘した問題案件の改善されず。( 9.5.0 地図は表示される様になった、でも、...,2024-10-29T06:47:35.118Z,いつもアプリをお使いいただき、ありがとうございます！\n地図のリストをタップしても非表示にな...,9.6.2,1,[{'criteria': 'vaf_never_display_disruptive_ad...,-,- 指摘した問題が改善されていない,[],"[指摘, し, 問題, 改善, さ, れ, い]"
2,2bd1806d-3415-4ac6-9db2-70800303fc48,酒井雄太,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-22T21:54:44.660Z,4,4,https://play.google.com/store/apps/details?id=...,NaN,Wi-Fiがある場所で自動的に接続できるので便利だと思います。,NaN,NaN,NaN,0,[],Wi-Fiがある場所で自動的に接続できるので便利。,-,"[Wi-Fi, ある, 場所, 自動的, 接続, できる, 便利]",[]
3,62bedb1c-1dd6-4807-8544-a04ceda38826,菅原金二郎,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:53:26.341Z,2,2,https://play.google.com/store/apps/details?id=...,NaN,CMが多い ５秒から30秒動画が流れます,NaN,NaN,9.6.2,0,[],-,CMが多く、5秒から30秒の動画が流れる,[],"[CM, 多く, 5秒, 30秒, 動画, 流れる]"
4,f67f4c45-f19c-4f7b-b792-6c703bb05915,みかん,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:23:41.626Z,5,5,https://play.google.com/store/apps/details?id=...,NaN,モバイルスマホを使っていますが、ネット回線が無くてもGMOWiFiで問題なく 検索やネット回...,2024-11-25T01:08:32.912Z,いつもアプリをお使いいただき、ありがとうございます！\n\n外出時のWiFi利用にお役立てい...,9.6.2,0,[],ネット回線が無くてもGMOWiFiで問題なく検索やネット回線が使用できるところ。,-,"[ネット, 回線, 無く, GMOWiFi, 問題, 検索, ネット, 回線, 使用, でき...",[]


In [11]:
# データを保存

df.to_csv('/Users/manqueenmannequin/TW_otameshi/project_root/data/processed/android_cleaned_mecab.csv', index=False)